In [384]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from string import punctuation
import re
from utils import *

In [385]:
df_bernardo = pd.read_csv('data/bernardoArevalo.csv')
df_sandra = pd.read_csv('data/sandraTorres.csv')

In [388]:
# Limpieza de datos

df_bernardo = df_bernardo.drop(df_bernardo.columns[0], axis=1)	# Primera columna no es necesaria
df_bernardo = df_bernardo.apply(lambda x: x.astype(str).str.lower())	# convertir todo a minúscula
df_bernardo = df_bernardo.drop(['id'], axis=1)
df_bernardo = df_bernardo.drop(['id_str'], axis=1)
df_bernardo = df_bernardo.drop(['url'], axis=1)

# Eliminar signos de puntuacion
df_bernardo['rawContent'] = df_bernardo['rawContent'].apply(lambda x: ''.join([i for i in x if i not in punctuation]))

# Eliminar @
df_bernardo['rawContent'] = df_bernardo['rawContent'].str.replace('@', '')
df_bernardo['rawContent'] = df_bernardo['rawContent'].str.replace('#', '')

# Eliminar emoji
df_bernardo['rawContent'] = df_bernardo['rawContent'].apply(lambda x: emoji_pattern.sub(r'', x))
df_bernardo['rawContent'] = df_bernardo['rawContent'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

df_bernardo = df_bernardo[df_bernardo['lang'] == 'es']
df_bernardo.head()

,date,user,lang,rawContent,replyCount,retweetCount,likeCount,quoteCount,conversationId,hashtags,cashtags,mentionedUsers,links,viewCount,retweetedTweet,quotedTweet,place,coordinates,inReplyToTweetId,inReplyToUser,source,sourceUrl,sourceLabel,media,_type
0,2023-09-13 23:58:52+00:00,"{'id': 46561608, 'id_str': '46561608', 'url': ...",es,anongtreloaded msemillagt barevalodeleon jajaj...,0,0,2,0,1702088813042954628,[],[],"[{'id': 1419402501212811265, 'username': 'anon...",[],184.0,nan,nan,nan,nan,1.7020888130429548e+18,"{'id': 1419402501212811265, 'username': 'anong...","<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,twitter web app,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.tweet
1,2023-09-13 23:58:24+00:00,"{'id': 1005627722, 'id_str': '1005627722', 'ur...",es,asiervera aztecanoticiagt barevalodeleon repit...,0,1,4,0,1702055546247885225,[],[],"[{'id': 291888009, 'username': 'asiervera', 'd...",[],107.0,nan,nan,nan,nan,1.7020555462478853e+18,"{'id': 291888009, 'username': 'asiervera', 'di...","<a href=""http://twitter.com/#!/download/ipad"" ...",http://twitter.com/#!/download/ipad,twitter for ipad,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.tweet
2,2023-09-13 16:26:49+00:00,"{'id': 743252657083650048, 'id_str': '74325265...",es,paciente 39 años dolor lumbar 1 año caída 45 s...,2,17,91,0,1701995859229958189,[],[],[],[],47277.0,nan,nan,nan,nan,nan,nan,"<a href=""https://help.twitter.com/en/using-twi...",https://help.twitter.com/en/using-twitter/how-...,advertiser-interface,{'photos': [{'url': 'https://pbs.twimg.com/med...,snscrape.modules.twitter.tweet
3,2023-09-13 23:56:55+00:00,"{'id': 1512621569834688513, 'id_str': '1512621...",es,viczacariasgt soy502 barevalodeleon msemillagt...,0,0,0,0,1701791008789250545,[],[],"[{'id': 84055675, 'username': 'viczacariasgt',...",[],nan,nan,nan,nan,nan,1.7017994428971622e+18,"{'id': 84055675, 'username': 'viczacariasgt', ...","<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,twitter for android,{'photos': [{'url': 'https://pbs.twimg.com/med...,snscrape.modules.twitter.tweet
4,2023-09-13 23:56:10+00:00,"{'id': 891058586, 'id_str': '891058586', 'url'...",es,igorbitkov sigan juego ruso invasor enero soli...,1,0,5,0,1701898230408057163,[],[],"[{'id': 707086237753090048, 'username': 'igor_...",[],69.0,nan,nan,nan,nan,1.701898230408057e+18,"{'id': 707086237753090048, 'username': 'igor_b...","<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,twitter for android,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.tweet


In [391]:

# Limpieza de datos

df_sandra = df_sandra.drop(df_sandra.columns[0], axis=1)	# Primera columna no es necesaria
df_sandra = df_sandra.apply(lambda x: x.astype(str).str.lower())	# convertir todo a minúscula
if 'id' in df_sandra.columns:
    df_sandra = df_sandra.drop(['id'], axis=1)
if 'id_str' in df_sandra.columns:
    df_sandra = df_sandra.drop(['id_str'], axis=1)
df_sandra = df_sandra.drop(['url'], axis=1)

# Eliminar signos de puntuacion
df_sandra['rawContent'] = df_sandra['rawContent'].apply(lambda x: ''.join([i for i in x if i not in punctuation]))

# Eliminar @
df_sandra['rawContent'] = df_sandra['rawContent'].str.replace('@', '')
df_sandra['rawContent'] = df_sandra['rawContent'].str.replace('#', '')

# Eliminar emoji
df_sandra['rawContent'] = df_sandra['rawContent'].apply(lambda x: emoji_pattern.sub(r'', x))
df_sandra['rawContent'] = df_sandra['rawContent'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

df_sandra = df_sandra[df_sandra['lang'] == 'es']
match = lambda user_str: re.search(r"'username':\s*'([^']+)'", user_str)
username_from_str = lambda user_str: match(user_str).group(1) if match(user_str) else None
df_sandra['direct_username'] = df_sandra['user'].apply(username_from_str)





In [392]:
# show all columns
pd.set_option('display.max_columns', None)

df_sandra.head()

,date,user,lang,rawContent,replyCount,retweetCount,likeCount,quoteCount,conversationId,hashtags,cashtags,mentionedUsers,links,viewCount,retweetedTweet,quotedTweet,place,coordinates,inReplyToTweetId,inReplyToUser,source,sourceUrl,sourceLabel,media,_type,direct_username
0,2023-09-12 19:56:32+00:00,"{'id': 1213902343311245313, 'id_str': '1213902...",es,bernardosilvagt barevalodeleon drgiammattei me...,0,0,0,0,1701683348756730305,[],[],"[{'id': 3337026291, 'username': 'bernardosilva...",[],10.0,nan,nan,nan,nan,1.7016833487567304e+18,"{'id': 3337026291, 'username': 'bernardosilvag...","<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,twitter for android,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.tweet,yeya16155804
1,2023-09-12 19:55:48+00:00,"{'id': 1420560045323038720, 'id_str': '1420560...",es,awskl mjcabrerar barevalodeleon tseguatemala d...,0,0,0,0,1701648306605117950,[],[],"[{'id': 390988524, 'username': '_awskl', 'disp...",[],23.0,nan,nan,nan,nan,1.7016848890813071e+18,"{'id': 390988524, 'username': '_awskl', 'displ...","<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,twitter web app,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.tweet,palomin17772524
3,2023-09-12 19:51:07+00:00,"{'id': 390988524, 'id_str': '390988524', 'url'...",es,palomin17772524 mjcabrerar barevalodeleon tseg...,1,0,0,0,1701648306605117950,[],[],"[{'id': 1420560045323038720, 'username': 'palo...",[],30.0,nan,nan,nan,nan,1.7016844593035226e+18,"{'id': 1420560045323038720, 'username': 'palom...","<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,twitter for android,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.tweet,_awskl
4,2023-09-12 19:50:27+00:00,"{'id': 454924874, 'id_str': '454924874', 'url'...",es,asolaresm mike051270 barevalodeleon estupido c...,0,0,0,0,1701328136997109939,[],[],"[{'id': 310913844, 'username': 'asolaresm', 'd...",[],4.0,nan,nan,nan,nan,1.701329458463138e+18,"{'id': 310913844, 'username': 'asolaresm', 'di...","<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,twitter for iphone,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.tweet,mr_andrew89
5,2023-09-12 19:46:53+00:00,"{'id': 390988524, 'id_str': '390988524', 'url'...",es,mjcabrerar palomin17772524 barevalodeleon tseg...,3,0,0,0,1701648306605117950,[],[],"[{'id': 15400237, 'username': 'mjcabrerar', 'd...",[],135.0,nan,nan,nan,nan,1.701648306605118e+18,"{'id': 15400237, 'username': 'mjcabrerar', 'di...","<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,twitter for android,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.tweet,_awskl


In [393]:
# count how many none values are in the username column
df_sandra['username'].isna().sum()

KeyError: 'username'

In [ ]:
print(df_sandra.shape)